TF

In [1]:
import seismic_ml_kernel

In [2]:
#train (train_data_dir, validation_data_dir, image_size)
#test (validation_data_dir + 'noise/', image_size)

In [3]:
from keras.datasets import mnist # subroutines for fetching the MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data() # fetch MNIST data

#mn (X_train, y_train, X_test, y_test)

Using TensorFlow backend.


In [4]:
from keras.datasets import cifar10 # subroutines for fetching the CIFAR-10 dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data() # fetch CIFAR-10 data

#ci (X_train, y_train, X_test, y_test)

In [1]:
def seis_dataset ():
    import numpy as np
    from seismic_handler import SeismicPrestack
    
    images_dir = 'images/'

    import os
    if os.path.exists(images_dir):
        import shutil
        shutil.rmtree(images_dir)

    if not os.path.exists(images_dir):
        os.makedirs(images_dir)

    train_data_dir = images_dir + 'train/'
    validation_data_dir = images_dir + 'validation/'

    if not os.path.exists(train_data_dir):
        os.makedirs(train_data_dir)

    if not os.path.exists(validation_data_dir):
        os.makedirs(validation_data_dir)
    
    handler = SeismicPrestack('data/prestack.sgy', False)
    handlerNoise = SeismicPrestack('data/prestack.sgy', True)
    
    from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

    X_train = []
    y_train = []
    X_test = [] 
    y_test = []
    
    counter = 0
    
    vals = handler.getHeaderVals ('original_field_record_number')
    valsNoise = handlerNoise.getHeaderVals ('original_field_record_number')
            
    for v in vals:

        parts = handler.readGatherParts (v, 50, 0.5)
        partsNoise = handlerNoise.readGatherParts (v, 50, 0.5)
        if len (partsNoise) != len (parts):
            exit ()
        
        nparts = len(parts)

        sh = parts[0].shape
        for i in range(nparts):
            #if sh != partsNoise[i].shape or sh != parts[i].shape:
            #    continue
            if i % 2 == 0:
                data = partsNoise[i]
                #diff = data - parts[i]
                #png_name = images_dir + str(counter) + '.png'
                #image = SeismicPrestack.convert_to_image (diff.T)
                #image.save (png_name)
                
                #data = SeismicPrestack.addNoiseStatic(data)
                category = 1
            else:
                category = 0
                data = parts[i]
                
            if data.shape != sh:
                #print ('shape error')
                continue

            image = SeismicPrestack.convert_to_image (data.T)
            data = img_to_array (image)

            if i < nparts*0.8:
                X_train.append(data)
                y_train.append (category)
                png_name = train_data_dir
            else:
                X_test.append(data)
                y_test.append(category)   
                png_name = validation_data_dir
                
            if category == 1:
                png_name += 'noise/'
            else:
                png_name += 'norm/'    
            
            if not os.path.exists(png_name):
                os.makedirs(png_name)
                
            png_name += str(counter) + '.png'
            #image.save (png_name)
                
            counter = counter+1
    
    y_test = np.array(y_test)
    y_train = np.array(y_train)
    X_train = np.array(X_train)
    X_test = np.array(X_test)
    
    
    if len(X_train) != len(y_train):
        print ('len(X_train) != len(y_train)')
        return
    if len(X_test) != len(y_test):
        print ('len(X_test) != len(y_test)')
        return

    return (X_train, y_train), (X_test, y_test)

In [ ]:
import seismic_ml_kernel

(X_train, y_train), (X_test, y_test) = seis_dataset ()

seismic_ml_kernel.ci (X_train, y_train, X_test, y_test)

Using TensorFlow backend.
C:\Users\kostyad\Documents\Jupyter\seismic_fingerprint\seismic_ml_kernel.py:78: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", padding="same")`
  conv_1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='relu')(inp)
C:\Users\kostyad\Documents\Jupyter\seismic_fingerprint\seismic_ml_kernel.py:79: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", padding="same")`
  conv_2 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='relu')(conv_1)
C:\Users\kostyad\Documents\Jupyter\seismic_fingerprint\seismic_ml_kernel.py:83: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", padding="same")`
  conv_3 = Convolution2D(conv_depth_2, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_1)
C:\Users\kostyad\Documents\Jupyter\seismic_fingerprint\s

Train on 4819 samples, validate on 536 samples
Epoch 1/200
4819/4819 [==============================] - 165s - loss: 0.7113 - acc: 0.5038 - val_loss: 0.6929 - val_acc: 0.5112
Epoch 2/200
4819/4819 [==============================] - 169s - loss: 0.6939 - acc: 0.5148 - val_loss: 0.6925 - val_acc: 0.5112
Epoch 3/200
4819/4819 [==============================] - 171s - loss: 0.6873 - acc: 0.5171 - val_loss: 0.6879 - val_acc: 0.5149
Epoch 4/200
4819/4819 [==============================] - 175s - loss: 0.6917 - acc: 0.5163 - val_loss: 0.6875 - val_acc: 0.5205
Epoch 5/200
4819/4819 [==============================] - 173s - loss: 0.6899 - acc: 0.5082 - val_loss: 0.6910 - val_acc: 0.5149
Epoch 6/200
4819/4819 [==============================] - 172s - loss: 0.6741 - acc: 0.5327 - val_loss: 0.6885 - val_acc: 0.5205
Epoch 7/200
4819/4819 [==============================] - 173s - loss: 0.6895 - acc: 0.5142 - val_loss: 0.6856 - val_acc: 0.5224
Epoch 8/200
4819/4819 [==============================] - 

KeyboardInterrupt: 